# Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

Cloud storage.

In [ ]:
!mkdir -p "$HOME/.lamin/cache"
!lamindb configure \
    --storage "s3://lamin0" \
    --cache "$HOME/.lamin/cache"

In [ ]:
from lamindb._secrets import NOTION_API_KEY
from os import environ
from lamindb._configuration import cloud_storage, storage_root, cache_root

assert cloud_storage
assert storage_root == "s3://lamin0"
assert cache_root == environ["HOME"] + "/.lamin/cache"

Local storage.

In [ ]:
!mkdir -p "$HOME/data"
!lamindb configure \
    --storage "$HOME/data"

In [ ]:
from lamindb._secrets import NOTION_API_KEY
from os import environ
from lamindb._configuration import cloud_storage, storage_root, cache_root

assert cloud_storage == False
assert storage_root == environ["HOME"] + "/data"
assert cache_root == ""